# Assignment: AI Travel Agent & Expense Planner

**Purpose:**  
Trip planning for any city worldwide with real-time data.

---

## Requirements

- Real-time weather information  
- Top attractions and activities  
- Hotel cost calculation (per day × total days)  
- Currency conversion to user's native currency  
- Complete itinerary generation  
- Total expense calculation  
- Generate a summary of the entire output  

---

## Agentic Workflow

user_input  
  |  
search attraction and activity  
1. search attracation  
2. search restaurant  
3. search activity  
4. search transportation  
  |  
search weather forcasting  
1. get current weather  
2. get weather forcast  
  |  
search hotel costs  
1. search hotel  
2. estimate the hotel cost  
3. budget_range  
  |  
calculate total cost  
1. add  
2. multiply  
3. calculated total cost  
4. calcualte the daily budget  
    |   
currency_converion  
1. get exchnage rate  
2. convert currancy  
    |   
Itinery generation  
1. get day plan  
2. crete full itinery  
    |  
create Trip Summary  
    |  
Retun complete traval plan  

---

## OOP Design Guideline

Note: if you know the OOPS then design this entire system using object and class in modular fashion.

---

## Submission Instructions

- **Deadline:** Next Friday 9PM IST
- **Submission Form:** https://forms.gle/g8RZ4qx8yvNcih4B7
- **Requirement:**  
  - Submit one GitHub link containing all your assignments.
  - Make sure your repository is well-organized and includes all code and

In [3]:
# Library imports
import operator
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from pydantic import BaseModel,Field
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.tools import tool

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()
import os

## Configure the model

In [4]:
# Set the environment variable for the Groq API key
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

# Import the necessary model from langchain_groq
from langchain_groq import ChatGroq
model = ChatGroq(model_name="deepseek-r1-distill-llama-70b", temperature=0.1)

# from langchain_google_genai import ChatGoogleGenerativeAI
# model=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

output=model.invoke("What is the capital of France?")
print(output.content)

<think>

</think>

The capital of France is Paris.


## importing the inbuilt tools

In [13]:
# for searching attractions, Restaurants, and hotels
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
search.invoke("What is the capital of France?")

# # for weather information
# from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun 
# weather = OpenWeatherMapQueryRun()
# weather.invoke("What is the weather in Paris?")

# # for currency conversion
# from langchain_community.tools.currency_exchange import CurrencyExchangeAPIWrapper
# currency_exchange = CurrencyExchangeAPIWrapper()
# currency_exchange.invoke("Convert 100 USD to EUR")

[{'title': 'Paris - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/Paris',
  'content': 'As the capital of France, Paris is the seat of France\'s [national government](https://en.wikipedia.org/wiki/Government_of_France "Government of France"). For the executive, the two chief officers each have their own official residences, which also serve as their offices. The [President of the French Republic](https://en.wikipedia.org/wiki/President_of_France "President of France") resides at the [Élysée Palace](https://en.wikipedia.org/wiki/%C3%89lys%C3%A9e_Palace "Élysée [...] "Capital city") and [largest city](https://en.wikipedia.org/wiki/List_of_communes_in_France_with_over_20,000_inhabitants "List of communes in France with over 20,000 inhabitants") of [France](https://en.wikipedia.org/wiki/France "France"). With an estimated population of 2,048,472 residents in January 2025[[3]](https://en.wikipedia.org/wiki/Paris#cite_note-pop2025-3) in an area of more than 105 km 2 (41 sq mi),[[4]](ht

In [1]:
# ...existing code...
class AttractionSearcher:
    def search_attractions(self, city): ...
    def search_restaurants(self, city): ...
    def search_activities(self, city): ...
    def search_transportation(self, city): ...

class WeatherService:
    def get_current_weather(self, city): ...
    def get_weather_forecast(self, city, days): ...

class HotelCostEstimator:
    def search_hotels(self, city): ...
    def estimate_hotel_cost(self, hotel, days): ...
    def budget_range(self, min_budget, max_budget): ...

class ExpenseCalculator:
    def add(self, *args): ...
    def multiply(self, a, b): ...
    def calculate_total_cost(self, items): ...
    def calculate_daily_budget(self, total, days): ...

class CurrencyConverter:
    def get_exchange_rate(self, from_currency, to_currency): ...
    def convert_currency(self, amount, rate): ...

class ItineraryGenerator:
    def get_day_plan(self, city, day): ...
    def create_full_itinerary(self, city, days): ...

class TripSummary:
    def generate_summary(self, details): ...
# ...existing code...

In [2]:
# ...existing code...
from langgraph.graph import StateGraph

# Define your state (can be a dict or a dataclass)
class TripState(dict): pass

# Instantiate your modules
attraction_searcher = AttractionSearcher()
weather_service = WeatherService()
hotel_estimator = HotelCostEstimator()
expense_calculator = ExpenseCalculator()
currency_converter = CurrencyConverter()
itinerary_generator = ItineraryGenerator()
trip_summary = TripSummary()

# Create the graph
graph = StateGraph(TripState)

# Add nodes (functions/classes that operate on state)
graph.add_node("search_attractions", attraction_searcher.search_attractions)
graph.add_node("search_restaurants", attraction_searcher.search_restaurants)
graph.add_node("search_activities", attraction_searcher.search_activities)
graph.add_node("search_transportation", attraction_searcher.search_transportation)

graph.add_node("get_current_weather", weather_service.get_current_weather)
graph.add_node("get_weather_forecast", weather_service.get_weather_forecast)

graph.add_node("search_hotels", hotel_estimator.search_hotels)
graph.add_node("estimate_hotel_cost", hotel_estimator.estimate_hotel_cost)
graph.add_node("budget_range", hotel_estimator.budget_range)

graph.add_node("add", expense_calculator.add)
graph.add_node("multiply", expense_calculator.multiply)
graph.add_node("calculate_total_cost", expense_calculator.calculate_total_cost)
graph.add_node("calculate_daily_budget", expense_calculator.calculate_daily_budget)

graph.add_node("get_exchange_rate", currency_converter.get_exchange_rate)
graph.add_node("convert_currency", currency_converter.convert_currency)

graph.add_node("get_day_plan", itinerary_generator.get_day_plan)
graph.add_node("create_full_itinerary", itinerary_generator.create_full_itinerary)

graph.add_node("generate_summary", trip_summary.generate_summary)

# Define the flow (edges)
graph.set_entry_point("search_attractions")
graph.add_edge("search_attractions", "search_restaurants")
graph.add_edge("search_restaurants", "search_activities")
graph.add_edge("search_activities", "search_transportation")
graph.add_edge("search_transportation", "get_current_weather")
graph.add_edge("get_current_weather", "get_weather_forecast")
graph.add_edge("get_weather_forecast", "search_hotels")
graph.add_edge("search_hotels", "estimate_hotel_cost")
graph.add_edge("estimate_hotel_cost", "budget_range")
graph.add_edge("budget_range", "add")
graph.add_edge("add", "multiply")
graph.add_edge("multiply", "calculate_total_cost")
graph.add_edge("calculate_total_cost", "calculate_daily_budget")
graph.add_edge("calculate_daily_budget", "get_exchange_rate")
graph.add_edge("get_exchange_rate", "convert_currency")
graph.add_edge("convert_currency", "get_day_plan")
graph.add_edge("get_day_plan", "create_full_itinerary")
graph.add_edge("create_full_itinerary", "generate_summary")
graph.add_edge("generate_summary", END)  # END is the terminal node

# Compile the graph
app = graph.compile()
# ...existing code...


NameError: name 'END' is not defined

In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))